# Increase the speed by reducing the amount of data

In [ ]:
import holoviews as hv

from postopus.octopus_run import Run

hv.extension("bokeh", "matplotlib")

In [ ]:
path_methane = "../tests/data/methane/"
run = Run(path_methane)

In [ ]:
xat = run.default.scf.density.get_all(source="vtk")

In [ ]:
hv_dst = hv.Dataset(xat)
hv_imt = hv_dst.to(hv.Image, dynamic=True)
hv.output(max_frames=10000)

In [ ]:
hv_imt

We've never encountered such a situation but it is possible that even when setting `dynamic=True` the data points per frame is too large, i.e. too dense to be loaded quickly. In these cases, one can try to use the following methods to speed things up:

In [ ]:
# https://holoviews.org/user_guide/Large_Data.html

In [ ]:
# for hv.HeatMap
from holoviews.operation import decimate

# for hv.Image
# shade, dynspread, spread
# datashade is the same as shade(rasterize(img))
from holoviews.operation.datashader import rasterize

In [ ]:
rasterize(hv_imt)  # Not a really curated plot, just to play around with sliders

All of these "reduction" methods have their own parameters, try to play around with them.
A good example is `decimate` and its `max_samples`. Try to change `max_samples` to 1000 and see what happens.
Remember that you can use `hv.help(<name_of_method>)` to see which parameters are available.

In [ ]:
# HeatMap
hv_dst = hv.Dataset(xat)
hv_hmt = hv_dst.to(hv.HeatMap)
decimate(hv_hmt, max_samples=10000)

# Extra: Comparison between decimate and full data

As one can see in this section, one can achieve arbitrary high precision with `decimate`, if the `max_samples` parameter is high enough.
One can create composed plots like in this example. More info in: http://holoviews.org/user_guide/Composing_Elements.html

In [ ]:
path_interference = "../tests/data/interference/"
run = Run(path_interference)

In [ ]:
xa = run.Maxwell.td.b_field.x.get_all(source="z=0")

In [ ]:
hv_dst = hv.Dataset(xa)
hv_imt = hv_dst.to(hv.Image, ["x", "y"])  # needed for generating outputs
hv_imt_dynamic = hv_dst.to(hv.Image, ["x", "y"], dynamic=True)
hv.output(max_frames=3000)
hv.output(widget_location="bottom")

In [ ]:
hv_dst = hv.Dataset(xa)
hv_hmt = hv_dst.to(hv.HeatMap, dynamic=True)
deci = decimate(hv_hmt, max_samples=500000000000).opts(
    colorbar=True, width=500, height=350, cmap="seismic", clim=(-(10**-3), 10**-3)
)  # in theory one can approach the exact image
# approximation (left) vs. exact image (right)
deci + hv_hmt.opts(
    colorbar=True, width=500, height=350, cmap="seismic", clim=(-(10**-3), 10**-3)
)

In the example above, we used a `max_samples` higher than the amount of data points in an image, so that the approximation is exact. Beneath we will show an example where the amount of data points is smaller than in the original image

In [ ]:
# Actual approximation
hv_dst = hv.Dataset(xa)
hv_hmt = hv_dst.to(hv.HeatMap, dynamic=True)
deci = decimate(hv_hmt, max_samples=100000).opts(
    colorbar=True, width=500, height=350, cmap="seismic", clim=(-(10**-3), 10**-3)
)
# approximation (left) vs. exact image (right)
deci + hv_hmt.opts(
    colorbar=True, width=500, height=350, cmap="seismic", clim=(-(10**-3), 10**-3)
)